# Semantic Kernel 

In this code sample, you will use the [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework to create a basic agent. 

The goal of this sample is to show you the steps that we will later use in the addtional code samples when implementing the different agentic patterns. 

## Import the Needed Python Packages 

In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv



from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory


from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

## Creating the Client and Kernel 

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `ai_model_id` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

For us to us the `Azure Inference SDK` that is used for the `base_url` for GitHub Models, we will use the `AsyncOpenAI` connector within Semantic Kernel. There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use Semantic Kernel for other model providers.

We will also create a `Kernel`. A `kernel` is a collection of the services and plugins that will be used by your Agents. In this snipppet, we are creating the kernel and adding the `chat_completion_service` to it.  

In [ ]:
import random   

# Define a sample plugin for the sample
class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    # The __init__ method you shared is indeed a constructor for the DestinationsPlugin class. 
    # In Python, __init__ serves as the class constructor and is automatically called when you
    # create a new instance of the class.
    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    # @kernel_function is a decorator from the Semantic Kernel framework that transforms a regular 
    # Python method into a function that can be discovered and called by AI agents.
    #   Registers the function with Semantic Kernel so it's available to agents
    #   Provides metadata about the function's purpose and behavior
    #   Enables function calling by the AI model through the kernel#
    @kernel_function(description="Provides a random vacation destination.") 
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]: 
        # An annotation is used to provide type hints (a string) and metadata (description) for the function's return value.
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
# load_dotenv() loads environment variables from .env file, 
#  making credentials available while keeping them secure and out of source code
load_dotenv()

# Creates AsyncOpenAI client to connect to Azure-hosted models
# Uses GITHUB_TOKEN from environment variables for authentication
# Points to Azure's inference endpoint via base_url
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), base_url="https://models.inference.ai.azure.com/")

# Instantiates the Kernel - the central orchestration component
# Registers DestinationsPlugin with a logical name ("destinations") for function discovery
kernel = Kernel()
kernel.add_plugin(DestinationsPlugin(), plugin_name="destinations")

# Defines service_id ("agent") to identify a specific LM service instance. Doing so 
# enables the system to reference the LM service elsewhere in the code.
service_id = "agent"

#   Creates instance of OpenAIChatCompletion, which acts as a service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini", # gpt-4o-mini as the model
    async_client=client, # previously created async client
    service_id=service_id # creates OpenAIChatCompletion service using:
)

# Adds the service to the kernel, making it available to the agent
kernel.add_service(chat_completion_service)

## Creating the Agent 

Below we will are creating the Agent called `TravelAgent` and also creating a variable called `AGENT_INSTRUCTIONS`. We will later add this to our `system_message` that will give the agent instructions on the task, behavior and tone.

For this example, we are using very simple instructions. You can change these instructions to see how the agent responds differently. 

In [ ]:
# This configuration is what enables the natural function calling seen later when users ask about 
# travel destinations. The agent autonomously decides when to use the destination function.
# This example, the user never needs to know the function exists - they just ask for travel plans, 
# and the agent intelligently uses its available tools.

# Fetches configuration settings for the specified service
# Uses the previously defined service_id ("agent") to retrieve the right settings
settings = kernel.get_prompt_execution_settings_from_service_id(
    service_id=service_id)

# Enables AI-driven function calling
# Instructs the LLM to automatically determine when to call functions (rather than requiring explicit instructions)
# Allows the agent to independently decide when to use get_random_destination() based on conversation context
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# 'FunctionChoiceBehavior.Auto()' - This setting tells Semantic Kernel to let the LLM (in this case GPT-4o-mini) 
# decide when to call functions on its own, rather than requiring direct instructions. Here's what this means: 
#
# Without Auto Function Calling:
#  Developer would need to explicitly code when to call functions
#  Or users would need to explicitly ask for specific functions ("get me a random destination")
#  Leads to unnatural interactions and more complex code
#
# With Auto Function Calling:
#  The AI model analyzes user requests like "Plan me a day trip"
#  It recognizes when a registered function would be helpful
#  It autonomously chooses to call get_random_destination() when appropriate
#  The kernel handles the actual function execution
#
#  This creates a more natural conversation flow where the travel agent can seamlessly integrate function outputs 
#  (destination suggestions) into its responses based on contextual understanding of the conversation.
#
#  In this example, the user never needs to know the function exists - they just ask for travel plans,
#  and the agent intelligently uses its available tools.

In [ ]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = "You are a helpful AI Agent that can help plan vacations for customers at random destinations"

# ChatCompletionAgent is a specialized SK agent that uses chat-based interactions to communicate with users.
# SK encapsulates the agent's behavior, instructions, and settings into a single object.
# This class creates an agent that can:
#   Process natural language through the connected language model
#   Call functions autonomously based on the conversation context
#   Maintain conversation state across multiple turns
#   Follow system instructions defined in AGENT_INSTRUCTIONS
agent = ChatCompletionAgent(
    service_id=service_id, 
    kernel=kernel, 
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings)
)

## Running the Agents 

Now we can run the Agent by defining the `ChatHistory` and adding the `system_message` to it. We will use the `AGENT_INSTRUCTIONS` that we defined earlier. 

After these are defined, we create a `user_inputs` that will be what the user is sending to the agent. In this case, we have set this message to `Plan me a sunny vacation`. 

Feel free to change this message to see how the agent responds differently. 

In [ ]:
from IPython.display import display, HTML  # Import utilities to render HTML in Jupyter notebooks

async def main():
    # --- CHAT HISTORY INITIALIZATION ---
    # External History Management: The history object is created and managed outside the agent
    #   Explicit Additions: You choose when to add user messages
    #   Pre-processing: You can modify the history before passing it to the agent
    #      You could implement a "Sliding Window with Decay" Pattern"
    #   Complete Replacement: You could substitute a custom history implementation
    chat_history = ChatHistory()  # Create empty conversation container
    
    # --- USER INPUT DEFINITION ---
    # Define multiple messages to simulate a multi-turn conversation
    # This tests both initial response and contextual follow-up handling
    user_inputs = [
        "Plan me a day trip.",  # Initial query that likely triggers destination function
        "I don't like that destination. Plan me another vacation.",  # Follow-up showing contextual awareness
    ]
    
    # --- PROCESS EACH USER MESSAGE ---
    # Iterate through each message to simulate a conversation
    for user_input in user_inputs:
        # Add current message to conversation history
        # This updates the context that will be sent to the LLM
        chat_history.add_user_message(user_input)  # Adds with "user" role automatically
        
        # --- HTML RENDERING SETUP (USER MESSAGE) ---
        # Start constructing the HTML display for this conversation turn
        # Create container div with bottom margin
        html_output = f"<div style='margin-bottom:10px'>"
        # Add bold "User:" label
        html_output += f"<div style='font-weight:bold'>User:</div>"
        # Add user's message with indentation for readability
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"  # Close the user message container
        
        # --- RESPONSE TRACKING VARIABLES ---
        agent_name: str | None = None  # Will store agent name when available
        full_response = ""  # Accumulates text portions of the response
        function_calls = []  # Tracks function call events and results
        function_results = {}  # Maps function names to their returned values
        
        # --- STREAM PROCESSING LOOP ---
        # Process chunks from the streaming response
        # Key to function calling: we process content incrementally as it arrives
        # Agent Invocation with the current conversation context through Streaming API
        
        # The 'async for' loop processes each chunk non-blockingly, which is essential when:
        #   Working with potentially slow network requests
        #   Handling function execution that might take time
        #   Maintaining responsiveness in interactive applications

        # The 'invoke_stream()' method returns an asynchronous iterator that:
        #   Produces response chunks as they become available from the LLM
        #   Allows for real-time processing of content without waiting for the complete response
        #   Enables immediate reaction to LLM decisions
        async for content in agent.invoke_stream(chat_history):
            # Extract agent name if available and not already captured
            if not agent_name and hasattr(content, 'name'):
                agent_name = content.name  # Store name for display
            
            # --- FUNCTION CALL DETECTION AND HANDLING ---
            # Process each content item to detect function calls and results
            for item in content.items:
                # CASE 1: LLM has decided to call a function
                if isinstance(item, FunctionCallContent):
                    # FunctionCallContent contains:
                    # - function_name: which registered function to call
                    # - arguments: parameters to pass to the function (as dict/JSON)
                    call_info = f"Calling: {item.function_name}({item.arguments})"
                    # Add call to tracking list for UI display
                    function_calls.append(call_info)
                     # The function execution happens HERE ↓ but is invisible in the code
                     # When Semantic Kernel sees FunctionCallContent, it:
                     #   1. Looks up the registered function ("destinations.get_random_destination")
                     #   2. Executes it with the provided arguments
                     #   3. Captures the return value
                     # Function execution happens automatically via kernel within the framework pipeline
                
                # CASE 2: Detects that a function has returned results
                elif isinstance(item, FunctionResultContent):
                    # FunctionResultContent contains:
                    # - function_name: which function produced this result
                    # - result: the actual return value from the function
                    # Format the function result for display
                    result_info = f"Result: {item.result}"
                    # Add the result to the tracking list
                    function_calls.append(result_info)
                    # Store result in dictionary for potential later use
                    # Key = function name, Value = return value
                    function_results[item.function_name] = item.result
                    # Behind the scenes:
                    # 1. Function was executed by SK framework
                    # 2. Result is captured and fed back to the LLM
                    # 3. LLM incorporates result into ongoing response generation
            
            # --- TEXT RESPONSE EXTRACTION ---
            # Process text content from the agent (not function-related messages)
            # Add text content to response if all conditions are met:
            if (hasattr(content, 'content') and  # Has content attribute
                content.content and  # Content is not None
                content.content.strip() and  # Content is not just whitespace
                # Content is not from function calls/results
                not any(isinstance(item, (FunctionCallContent, FunctionResultContent))
                      for item in content.items)):
                # Add this chunk to the accumulated response text
                full_response += content.content  # Append text to full response
        
        # --- FUNCTION CALL DISPLAY GENERATION ---
        # If any functions were called, create a collapsible UI section to show them
        if function_calls:
            html_output += f"<div style='margin-bottom:10px'>"
            html_output += f"<details>"  # HTML tag for expandable content
            # Create clickable summary header for the details
            html_output += f"<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
            # Create styled container for function call logs
            html_output += f"<div style='margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap;'>"
            # Join all function call logs with line breaks
            html_output += "<br>".join(function_calls)
            html_output += f"</div></details></div>"  # Close containers
        
        # --- AGENT RESPONSE DISPLAY GENERATION ---
        # Create container for agent's text response
        html_output += f"<div style='margin-bottom:20px'>"
        # Add agent name (or fallback to 'Assistant')
        html_output += f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
        # Add formatted agent response with indentation and whitespace preservation
        html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
        html_output += f"</div>"  # Close agent response container
        html_output += "<hr>"  # Add horizontal rule between conversation turns
        
        # --- RENDER OUTPUT ---
        # Convert HTML string to renderable HTML
        # Render the final HTML output in the Jupyter notebook
        display(HTML(html_output))  # Jupyter display function

# --- EXECUTE MAIN FUNCTION ---
# Call the async main function and await its completion
await main()

In [ ]:
from IPython.display import display, HTML

async def main():
    # Define the chat history
    # External History Management: The history object is created and managed outside the agent
    #   Explicit Additions: You choose when to add user messages
    #   Pre-processing: You can modify the history before passing it to the agent
    #      You could implement a "Sliding Window with Decay" Pattern"
    #   Complete Replacement: You could substitute a custom history implementation
    chat_history = ChatHistory()

    # Respond to user input
    user_inputs = [
        "Plan me a day trip.",
        "I don't like that destination. Plan me another vacation.",
    ]

    for user_input in user_inputs:
        # Add the user input to the chat history
        chat_history.add_user_message(user_input)

        # Start building HTML output
        html_output = f"<div style='margin-bottom:10px'>"
        html_output += f"<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"

        agent_name: str | None = None
        full_response = ""
        function_calls = []
        function_results = {}

        # Collect the agent's response with function call tracking
        async for content in agent.invoke_stream(chat_history):
            if not agent_name and hasattr(content, 'name'):
                agent_name = content.name

            # Track function calls and results
            for item in content.items:
                if isinstance(item, FunctionCallContent):
                    call_info = f"Calling: {item.function_name}({item.arguments})"
                    function_calls.append(call_info)
                elif isinstance(item, FunctionResultContent):
                    result_info = f"Result: {item.result}"
                    function_calls.append(result_info)
                    # Store function results
                    function_results[item.function_name] = item.result

            # Add content to response if it's not a function-related message
            if (hasattr(content, 'content') and content.content and content.content.strip() and
                not any(isinstance(item, (FunctionCallContent, FunctionResultContent))
                        for item in content.items)):
                full_response += content.content

        # Add function calls to HTML if any occurred
        if function_calls:
            html_output += f"<div style='margin-bottom:10px'>"
            html_output += f"<details>"
            html_output += f"<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
            html_output += f"<div style='margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap;'>"
            html_output += "<br>".join(function_calls)
            html_output += f"</div></details></div>"

        # Add agent response to HTML
        html_output += f"<div style='margin-bottom:20px'>"
        html_output += f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
        html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
        html_output += f"</div>"
        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()